#### general_search
The general search is exactly the code in the lecture slides. The only modification is the expanded nodes count and filtering visited nodes

<img src="images/general_search.png" alt="drawing" width="450" align="left"/>

In [ ]:
def general_search(problem, queueing_function):
    nodes = [Node(problem.initial_state)]
    visited = list()         # modification: filter visited nodes
    count = 0                # modicication: return number of expanded nodes
    while nodes:
        node = nodes.pop(0)  # remove front
        visited.append(node)
        if problem.goal_test(node.state):
            return node, count
        nodes = queueing_function(nodes, node.expand(problem))
        nodes = filter_visited(nodes, visited)
        count += 1
    return None, count

def filter_visited(nodes, visited):
    return list(set(nodes) - set(visited))


#### breadth first search

uses the general_search but utlisies the `enqueue_at_end` queueing strategy which appends the expanded nodes to the back of the node queue

In [ ]:
# ##########################
# Breadth First Search (BFS)
# ##########################


def breadth_first_search(problem):
    return general_search(problem, enqueue_at_end)


def enqueue_at_end(nodes, expanded_nodes):
    nodes.extend(expanded_nodes)
    return nodes

#### depth first search

uses the general_search but utlisies the `enqueue_at_front` queueing strategy which appends the expanded nodes to the front of the node queue

In [ ]:
# ########################
# Depth First Search (DFS)
# ########################


def depth_first_search(problem):
    return general_search(problem, enqueue_at_front)


def enqueue_at_front(nodes, expanded_nodes):
    expanded_nodes.extend(nodes)
    return expanded_nodes

#### iterative_deepening_search

uses the general_search in a loop but utlisies the `enqueue_at_front_cutoff` queueing strategy which appends the expanded nodes to the front of the node queue if their depth is less than the cut off value

In [ ]:
# ########################
# Iterative Deepening (ID)
# ########################


def iterative_deepening_search(problem):
    total_count = 0
    for i in itertools.count():
        solution, count = depth_limited_search(problem, i)
        total_count += total_count
        if solution:
            return solution, total_count


def depth_limited_search(problem, depth):
    cutoff_queue_func = functools.partial(enqueue_at_front_cutoff, depth)
    return general_search(problem, cutoff_queue_func)


def enqueue_at_front_cutoff(depth, nodes, expanded_nodes):
    nodes = enqueue_at_front(nodes, expanded_nodes)
    return [x for x in nodes if x.depth <= depth]

#### uniform_cost_search

uses the general_search but utlisies the `ordered_insert` queueing strategy which appends the expanded nodes to the node queue then sorts it

In [ ]:
# ########################
# Uniform Cost Search (UC)
# ########################


def uniform_cost_search(problem):
    return general_search(problem, ordered_insert)


def ordered_insert(nodes, expanded_nodes):
    nodes.extend(expanded_nodes)
    ordered_nodes = sorted(nodes, key=operator.attrgetter('path_cost'))
    return ordered_nodes

#### greedy_search

uses the general_search but utlisies the `ordered_eval_insert` queueing strategy which appends the expanded nodes to the node queue then sorts it according to the `eval_function`

In [ ]:
# ##################
# Greedy Search (GC)
# ##################


def greedy_search_1(problem):
    return best_first_search(problem, heuristic_1)


def greedy_search_2(problem):
    return best_first_search(problem, heuristic_2)


def greedy_search_3(problem):
    return best_first_search(problem, heuristic_3)


def best_first_search(problem, eval_function):
    heuristic_queue_func = functools.partial(ordered_eval_insert, eval_function)
    return general_search(problem, heuristic_queue_func)


def ordered_eval_insert(eval_function, nodes, expanded_nodes):
    nodes.extend(expanded_nodes)
    heuristic_nodes = [(node, eval_function(node)) for node in nodes]
    ordered_nodes = sorted(heuristic_nodes, key=lambda x: x[1])
    return [x for x, y in ordered_nodes]

#### heuristic definitions

1. **first heuristic:**
returns the distance to the closest white walker/dragon stone

2. **second heuristic:**
returns the sum of all distances to the white walkers

3. **third heuristic:**
returns the number of white walkers remaining

In [ ]:
def heuristic_1(node):
    """returns smallest distance between white walkers or dragon stone and current position"""
    x_node, y_node = node.state.location()
    goals = node.state.grid.components.white_walkers
    goals.append(node.state.grid.components.dragon_stone)
    distance = [np.sqrt((x_node - x)**2 + (y_node - y)**2) for x, y in goals]
    return distance[np.argmin(distance)]


def heuristic_2(node):
    """returns sum of distances between white walkers and current position"""
    x_node, y_node = node.state.location()
    goals = node.state.grid.components.white_walkers
    distance = [np.sqrt((x_node - x)**2 + (y_node - y)**2) for x, y in goals]
    return np.sum(distance)


def heuristic_3(node):
    """number of white_walkers"""
    return node.state.white_walkers()

#### a_star

uses the general_search but utlisies the `ordered_eval_insert` queueing strategy which appends the expanded nodes to the node queue then sorts it according to the `eval_function` + `path_cost`

In [ ]:
# ##################
# A Star Search (AC)
# ##################


def a_star_1(problem):
    return best_first_search(problem, a_heuristic_1)


def a_star_2(problem):
    return best_first_search(problem, a_heuristic_2)


def a_star_3(problem):
    return best_first_search(problem, a_heuristic_3)

In [ ]:
def a_heuristic_1(node):
    return node.path_cost + heuristic_1(node)


def a_heuristic_2(node):
    return node.path_cost + heuristic_2(node)


def a_heuristic_3(node):
    return node.path_cost + heuristic_3(node)